# 📊 Análise Jurimétrica - Workflow Completo

Este notebook demonstra o workflow completo de análise jurimétrica, desde a geração/carregamento de dados até a geração de relatórios.

## Índice

1. [Configuração e Importações](#1-configuração-e-importações)
2. [Geração/Carregamento de Dados](#2-geraçãocarregamento-de-dados)
3. [Análise Exploratória](#3-análise-exploratória)
4. [Análise Descritiva](#4-análise-descritiva)
5. [Análise Preditiva](#5-análise-preditiva)
6. [Análise de Sobrevivência](#6-análise-de-sobrevivência)
7. [Conclusões e Próximos Passos](#7-conclusões-e-próximos-passos)

## 1. Configuração e Importações

In [ ]:
# Importações básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Survival Analysis
from lifelines import KaplanMeierFitter, CoxPHFitter

warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✓ Bibliotecas importadas com sucesso!")

## 2. Geração/Carregamento de Dados

Vamos gerar dados simulados para demonstração. Em produção, você carregaria dados reais aqui.

In [ ]:
# Importar função de geração de dados
import sys
sys.path.append('..')
from jurimetria_completa import gerar_dados_simulados

# Gerar dados
df = gerar_dados_simulados(n_processos=200, seed=42)

print(f"Dataset gerado com {len(df)} processos")
print(f"\nPrimeiras linhas:")
df.head()

In [ ]:
# Informações sobre o dataset
print("Informações do Dataset:")
print(df.info())

print("\n" + "="*60)
print("Estatísticas Descritivas:")
print(df.describe())

## 3. Análise Exploratória

Vamos explorar os dados para entender suas características.

In [ ]:
# Verificar valores ausentes
print("Valores Ausentes:")
print(df.isnull().sum())

print("\n" + "="*60)
print("\nDistribuição por Tribunal:")
print(df['tribunal'].value_counts())

print("\n" + "="*60)
print("\nDistribuição por Classe Processual:")
print(df['classe'].value_counts())

print("\n" + "="*60)
print("\nDistribuição de Resultados:")
print(df['resultado'].value_counts())
print(f"\nTaxa de Procedência: {df['resultado'].mean()*100:.2f}%")

## 4. Análise Descritiva

### 4.1 Distribuição do Tempo de Tramitação

In [ ]:
# Histograma do tempo de tramitação
fig, ax = plt.subplots(figsize=(12, 6))
ax.hist(df['tempo_tramitacao_dias'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
ax.axvline(df['tempo_tramitacao_dias'].mean(), color='red', linestyle='--', linewidth=2, label=f'Média: {df["tempo_tramitacao_dias"].mean():.0f} dias')
ax.axvline(df['tempo_tramitacao_dias'].median(), color='green', linestyle='--', linewidth=2, label=f'Mediana: {df["tempo_tramitacao_dias"].median():.0f} dias')
ax.set_xlabel('Tempo de Tramitação (dias)', fontsize=12)
ax.set_ylabel('Frequência', fontsize=12)
ax.set_title('Distribuição do Tempo de Tramitação dos Processos', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 4.2 Análise por Juiz

In [ ]:
# Resultado por juiz
fig, ax = plt.subplots(figsize=(12, 6))
resultado_juiz = df.groupby(['juiz', 'resultado']).size().unstack(fill_value=0)
resultado_juiz.plot(kind='bar', stacked=False, ax=ax, color=['#e74c3c', '#2ecc71'])
ax.set_xlabel('Juiz', fontsize=12)
ax.set_ylabel('Quantidade de Processos', fontsize=12)
ax.set_title('Distribuição de Resultados por Juiz', fontsize=14, fontweight='bold')
ax.legend(['Improcedente', 'Procedente'], loc='upper right')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Taxa de procedência por juiz
taxa_por_juiz = df.groupby('juiz')['resultado'].mean() * 100
print("\nTaxa de Procedência por Juiz:")
print(taxa_por_juiz.sort_values(ascending=False))

### 4.3 Análise do Valor da Causa

In [ ]:
# Boxplot do valor da causa por resultado
fig, ax = plt.subplots(figsize=(10, 6))
df_plot = df.copy()
df_plot['resultado_label'] = df_plot['resultado'].map({0: 'Improcedente', 1: 'Procedente'})
sns.boxplot(data=df_plot, x='resultado_label', y='valor_causa', ax=ax, palette='Set2')
ax.set_xlabel('Resultado', fontsize=12)
ax.set_ylabel('Valor da Causa (R$)', fontsize=12)
ax.set_title('Distribuição do Valor da Causa por Resultado', fontsize=14, fontweight='bold')
ax.set_yscale('log')
plt.tight_layout()
plt.show()

# Estatísticas
print("\nValor Médio da Causa por Resultado:")
print(df.groupby('resultado')['valor_causa'].agg(['mean', 'median', 'std']))

### 4.4 Análise por Classe Processual

In [ ]:
# Tempo médio de tramitação por classe
tempo_por_classe = df.groupby('classe')['tempo_tramitacao_dias'].agg(['mean', 'median', 'std']).sort_values('mean', ascending=False)

fig, ax = plt.subplots(figsize=(10, 6))
tempo_por_classe['mean'].plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')
ax.set_xlabel('Classe Processual', fontsize=12)
ax.set_ylabel('Tempo Médio de Tramitação (dias)', fontsize=12)
ax.set_title('Tempo Médio de Tramitação por Classe Processual', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nEstatísticas de Tempo por Classe:")
print(tempo_por_classe)

## 5. Análise Preditiva

Vamos construir um modelo de regressão logística para prever o resultado de processos.

In [ ]:
# Preparação dos dados
df_model = df.copy()

# Criar variáveis dummy
df_model = pd.get_dummies(df_model, columns=['juiz', 'classe', 'tribunal'], drop_first=True)

# Criar faixas de valor
df_model['faixa_valor'] = pd.cut(
    df_model['valor_causa'],
    bins=[0, 10000, 50000, np.inf],
    labels=['Baixo', 'Médio', 'Alto']
)
df_model = pd.get_dummies(df_model, columns=['faixa_valor'], drop_first=True)

# Selecionar features
feature_cols = [col for col in df_model.columns if col.startswith(('juiz_', 'classe_', 'tribunal_', 'faixa_valor_', 'tempo_tramitacao'))]
X = df_model[feature_cols]
y = df_model['resultado']

print(f"Features selecionadas ({len(feature_cols)}):")
for col in feature_cols:
    print(f"  - {col}")

In [ ]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamanho do conjunto de treino: {len(X_train)}")
print(f"Tamanho do conjunto de teste: {len(X_test)}")

# Treinar modelo
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Predições
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

print("\n✓ Modelo treinado com sucesso!")

In [ ]:
# Avaliação do modelo
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=['Improcedente', 'Procedente']))

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, cbar=False)
ax.set_xlabel('Predito', fontsize=12)
ax.set_ylabel('Real', fontsize=12)
ax.set_title('Matriz de Confusão', fontsize=14, fontweight='bold')
ax.set_xticklabels(['Improcedente', 'Procedente'])
ax.set_yticklabels(['Improcedente', 'Procedente'])
plt.tight_layout()
plt.show()

In [ ]:
# Importância das features (Odds Ratios)
odds_ratios = np.exp(model.coef_[0])
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Odds Ratio': odds_ratios
}).sort_values('Odds Ratio', ascending=False)

fig, ax = plt.subplots(figsize=(10, 8))
feature_importance.set_index('Feature')['Odds Ratio'].plot(kind='barh', ax=ax, color='coral')
ax.axvline(x=1, color='red', linestyle='--', linewidth=2, alpha=0.5)
ax.set_xlabel('Odds Ratio', fontsize=12)
ax.set_title('Importância das Features (Odds Ratios)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nTop 5 Features mais importantes:")
print(feature_importance.head())

## 6. Análise de Sobrevivência

Vamos analisar o tempo até o encerramento dos processos usando Kaplan-Meier.

In [ ]:
# Curvas de Kaplan-Meier por classe processual
fig, ax = plt.subplots(figsize=(12, 7))

kmf = KaplanMeierFitter()

for classe in df['classe'].unique():
    mask = df['classe'] == classe
    kmf.fit(
        df.loc[mask, 'tempo_tramitacao_dias'],
        df.loc[mask, 'status'],
        label=classe
    )
    kmf.plot_survival_function(ax=ax, ci_show=True)

ax.set_xlabel('Tempo (dias)', fontsize=12)
ax.set_ylabel('Probabilidade de Sobrevivência\n(Processo ainda em andamento)', fontsize=12)
ax.set_title('Curva de Sobrevivência Kaplan-Meier por Classe Processual', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Modelo de Cox
df_cox = df.copy()
df_cox = pd.get_dummies(df_cox, columns=['juiz', 'classe', 'tribunal'], drop_first=True)

cph_cols = ['tempo_tramitacao_dias', 'status'] + [col for col in df_cox.columns if col.startswith(('juiz_', 'classe_', 'tribunal_'))]
df_cox_model = df_cox[cph_cols].copy()

cph = CoxPHFitter()
cph.fit(df_cox_model, duration_col='tempo_tramitacao_dias', event_col='status')

print("Modelo de Cox - Riscos Proporcionais:")
print(cph.summary[['coef', 'exp(coef)', 'p']])

## 7. Conclusões e Próximos Passos

### Principais Descobertas

1. **Tempo de Tramitação**: Observamos variação significativa no tempo de tramitação entre diferentes classes processuais.
2. **Fatores Preditivos**: O modelo de regressão logística identificou fatores importantes para prever o resultado dos processos.
3. **Análise de Sobrevivência**: As curvas de Kaplan-Meier mostram diferenças na duração dos processos por classe.

### Próximos Passos

1. **Integração com Dados Reais**: Substituir dados simulados por dados reais do DataJud
2. **Modelos Avançados**: Testar outros algoritmos (Random Forest, XGBoost)
3. **Feature Engineering**: Criar novas features baseadas em conhecimento do domínio
4. **Análise Temporal**: Investigar tendências ao longo do tempo
5. **Dashboard Interativo**: Criar dashboard com Plotly/Dash para visualização em tempo real

---

**Notebook criado para o Projeto Jurimetria**

Para mais informações, consulte o README.md do projeto.